In [24]:
# Jupyter notebook version of calibration

In [25]:
# from Yosef Meller's PBI

import shutil
import numpy as np
import numpy.random as rnd
from optv.calibration import Calibration
from optv.imgcoord import image_coordinates
from optv.transforms import convert_arr_metric_to_pixel
from optv.segmentation import target_recognition
from optv.imgcoord import image_coordinates
from optv.transforms import convert_arr_metric_to_pixel
from optv.orientation import match_detection_to_ref
from optv.orientation import external_calibration, full_calibration
from optv.calibration import Calibration
from optv.tracking_framebuf import TargetArray


from pyptv import ptv, parameter_gui, parameters as par

In [26]:
from pathlib import Path
par_path = Path("/home/user/Downloads/rbc300/parametersMultiPlain")

In [27]:


def get_pos(inters, R, angs):
    # Transpose of http://planning.cs.uiuc.edu/node102.html
    # Also consider the angles are reversed when moving from camera frame to
    # global frame.
    s = np.sin(angs)
    c = np.cos(angs)
    pos = inters + R*np.r_[ s[1], -c[1]*s[0], c[1]*c[0] ]
    return pos

def get_polar_rep(pos, angs):
    """
    Returns the point of intersection with zero Z plane, and distance from it.
    """
    s = np.sin(angs)
    c = np.cos(angs)
    zdir = -np.r_[ s[1], -c[1]*s[0], c[1]*c[0] ]
    
    c = -pos[2]/zdir[2]
    inters = pos + c*zdir
    R = np.linalg.norm(inters - pos)
    
    return inters[:2], R
    
def gen_calib(inters, R, angs, glass_vec, prim_point, radial_dist, decent):
    pos = get_pos(inters, R, angs)
    return Calibration(pos, angs, prim_point, radial_dist, decent, 
        np.r_[1, 0], glass_vec)

def fitness(solution, calib_targs, calib_detect, glass_vec, cpar):
    """
    Checks the fitness of an evolutionary solution of calibration values to 
    target points. Fitness is the sum of squares of the distance from each 
    guessed point to the closest neighbor.
    
    Arguments:
    solution - array, concatenated: position of intersection with Z=0 plane; 3 
        angles of exterior calibration; primary point (xh,yh,cc); 3 radial
        distortion parameters; 2 decentering parameters.
    calib_targs - a (p,3) array of p known points on the calibration target.
    calib_detect - a (d,2) array of d detected points in the calibration 
        target.
    cpar - a ControlParams object with image data.
    """
    # Breakdown of of agregate solution vector:
    inters = np.zeros(3)
    inters[:2] = solution[:2]
    R = solution[2]
    angs = solution[3:6] 
    prim_point = solution[6:9]
    rad_dist = solution[9:12]
    decent = solution[12:14]
        
    # Compare known points' projections to detections:
    cal = gen_calib(inters, R, angs, glass_vec, prim_point, rad_dist, decent)
    known_proj = image_coordinates(calib_targs, cal, 
        cpar.get_multimedia_params())
    known_2d = convert_arr_metric_to_pixel(known_proj, cpar)
    dists = np.linalg.norm(
        known_2d[None,:,:] - calib_detect[:,None,:], axis=2).min(axis=0)
    
    return np.sum(dists**2)


In [28]:
import os
working_folder = Path("/home/user/Downloads/rbc300")
working_folder.exists()



True

In [29]:
os.chdir(working_folder)
print(os.getcwd())


/home/user/Downloads/rbc300


In [30]:

calOriParams = par.CalOriParams(path = working_folder / "parameters")


In [40]:
def g(f):
    """ Returns a line without white spaces """
    return f.readline().strip()

def read(filepath, n_img = 4):
    with open(filepath, "r") as f:

        fixp_name = Path(g(f))
        fixp_name.exists()

        img_cal_name = []
        img_ori = []
        for i in range(n_img):
            img_cal_name.append(g(f))
            img_ori.append(g(f))

        tiff_flag = int(g(f)) != 0  # <-- overwrites the above
        pair_flag = int(g(f)) != 0
        chfield = int(g(f))


    # test if files are present, issue warnings
    for i in range(n_img):
        Path(img_cal_name[i]).exists()
        Path(img_ori[i]).exists()

    return fixp_name, img_cal_name, img_ori, tiff_flag, pair_flag, chfield


In [42]:
fixp_name, img_cal_name, img_ori, tiff_flag, pair_flag, chfield = read(working_folder / "parametersMultiPlane" / "cal_ori.par")
img_cal_name, img_ori

(['calibration_images/c0/calib_c0.tif',
  'calibration_images/c1/calib_c1.tif',
  'calibration_images/c2/calib_c2.tif',
  'calibration_images/c3/calib_c3.tif'],
 ['cal/cam1.tif.ori',
  'cal/cam2.tif.ori',
  'cal/cam3.tif.ori',
  'cal/cam4.tif.ori'])

In [37]:
list(calOriParams.path.rglob('*.par'))

[PosixPath('/home/user/Downloads/rbc300/parameters/multi_planes.par'),
 PosixPath('/home/user/Downloads/rbc300/parameters/criteria.par'),
 PosixPath('/home/user/Downloads/rbc300/parameters/track.par'),
 PosixPath('/home/user/Downloads/rbc300/parameters/detect_plate.par'),
 PosixPath('/home/user/Downloads/rbc300/parameters/sequence.par'),
 PosixPath('/home/user/Downloads/rbc300/parameters/shaking.par'),
 PosixPath('/home/user/Downloads/rbc300/parameters/man_ori.par'),
 PosixPath('/home/user/Downloads/rbc300/parameters/orient.par'),
 PosixPath('/home/user/Downloads/rbc300/parameters/targ_rec.par'),
 PosixPath('/home/user/Downloads/rbc300/parameters/examine.par'),
 PosixPath('/home/user/Downloads/rbc300/parameters/cal_ori.par'),
 PosixPath('/home/user/Downloads/rbc300/parameters/unsharp_mask.par'),
 PosixPath('/home/user/Downloads/rbc300/parameters/ptv.par'),
 PosixPath('/home/user/Downloads/rbc300/parameters/pft_version.par'),
 PosixPath('/home/user/Downloads/rbc300/parameters/dumbbell.p

In [ ]:
# for f in calOriParams.img_ori[:4]:
#     print(f"Backing up {f}")
#     shutil.copyfile(f, f + ".bck")
#     g = f.replace("ori", "addpar")
#     shutil.copyfile(g, g + ".bck")


In [ ]:

op = par.OrientParams()
op.read()

# recognized names for the flags:
names = [
    "cc",
    "xh",
    "yh",
    "k1",
    "k2",
    "k3",
    "p1",
    "p2",
    "scale",
    "shear",
]
op_names = [
    op.cc,
    op.xh,
    op.yh,
    op.k1,
    op.k2,
    op.k3,
    op.p1,
    op.p2,
    op.scale,
    op.shear,
]

flags = []
for name, op_name in zip(names, op_names):
    if op_name == 1:
        flags.append(name)

for i_cam in range(self.n_cams):  # iterate over all cameras

    if self.epar.Combine_Flag:

        self.status_text = "Multiplane calibration."
        """ Performs multiplane calibration, in which for all cameras the
        pre-processed planes in multi_plane.par combined.
        Overwrites the ori and addpar files of the cameras specified
        in cal_ori.par of the multiplane parameter folder
        """

        all_known = []
        all_detected = []

        for i in range(
            self.MultiParams.n_planes
        ):  # combine all single planes

            # c = self.calParams.img_ori[i_cam][-9] # Get camera id
            # not all ends with a number
            c = re.findall("\\d+", self.calParams.img_ori[i_cam])[0]

            file_known = (
                self.MultiParams.plane_name[i] + c + ".tif.fix"
            )
            file_detected = (
                self.MultiParams.plane_name[i] + c + ".tif.crd"
            )

            # Load calibration point information from plane i
            try:
                known = np.loadtxt(file_known)
                detected = np.loadtxt(file_detected)
            except BaseException:
                raise IOError(
                    "reading {} or {} failed".format(
                        file_known, file_detected
                    )
                )

            if np.any(detected == -999):
                raise ValueError(
                    (
                        "Using undetected points in {} will cause "
                        + "silliness. Quitting."
                    ).format(file_detected)
                )

            num_known = len(known)
            num_detect = len(detected)

            if num_known != num_detect:
                raise ValueError(
                    f"Number of detected points {num_known} does not match"
                    " number of known points {num_detect} for \
                        {file_known}, {file_detected}")

            if len(all_known) > 0:
                detected[:, 0] = (
                    all_detected[-1][-1, 0]
                    + 1
                    + np.arange(len(detected))
                )

            # Append to list of total known and detected points
            all_known.append(known)
            all_detected.append(detected)

        # Make into the format needed for full_calibration.
        all_known = np.vstack(all_known)[:, 1:]
        all_detected = np.vstack(all_detected)

        # this is the main difference in the multiplane mode
        # that we fill the targs and cal_points by the
        # combined information

        targs = TargetArray(len(all_detected))
        for tix in range(len(all_detected)):
            targ = targs[tix]
            det = all_detected[tix]

            targ.set_pnr(tix)
            targ.set_pos(det[1:])

        self.cal_points = np.empty((all_known.shape[0],)).astype(
            dtype=[("id", "i4"), ("pos", "3f8")]
        )
        self.cal_points["pos"] = all_known
    else:
        targs = self.sorted_targs[i_cam]


    residuals, targ_ix, err_est = full_calibration(
        self.cals[i_cam],
        self.cal_points["pos"],
        targs,
        self.cpar,
        flags,
    )